In [2]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
%matplotlib inline
import os
from os import listdir
import glob
import pickle
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import re
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import skimage
from skimage.io import imread
from skimage.io import imread,imshow
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Text dataset from labels

In [3]:
df = pd.read_csv(r"C:\Users\S H K\Downloads\project\labels.csv")

In [4]:
df["text_corrected"][4799]='im gonna build some fancy walls ven though i have millions of extra dollars in gonna make the mexicans pay for it'
df["text_corrected"][6781]='if donald and hillary are together on a boat in the middle of the ocean and it sinks who survives america'
df["text_corrected"][6784]='bruh whi this tub of margarine look like Donald Trump'
df["text_corrected"][6786]='2016 election trump vs hillary still a better love story than twilight'

In [5]:
#FILLING NAN VALUES WITH NONE
t = "none"
df.fillna(t, inplace = True)

In [6]:
length = len(df)
for i in range(length):
    if t in df["text_corrected"][i]:
        df["text_corrected"][i] = df["text_ocr"][i]  

In [7]:
def  normalize(df, col, new_col):
    df[new_col] = df[col].str.lower()
    df[new_col] = df[new_col].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_col] = df[new_col].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return df
data_clean = normalize(df, 'text_corrected', 'text_clean')

In [8]:
stop = stopwords.words('english')
data_clean['text_clean'] = data_clean['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [9]:
data_clean['text_clean'] = data_clean['text_clean'].apply(lambda x: word_tokenize(x))


In [10]:
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text
data_clean['text_clean'] = data_clean['text_clean'].apply(lambda x: word_lemmatizer(x))


In [11]:
var = data_clean["overall_sentiment"]
data_clean.pop("overall_sentiment")

0       very_positive
1       very_positive
2            positive
3            positive
4             neutral
            ...      
6987          neutral
6988          neutral
6989         positive
6990    very_positive
6991         positive
Name: overall_sentiment, Length: 6992, dtype: object

In [12]:
data_clean.insert(5, "overall_sentiment",var, True)   

In [13]:
#merging classes 
for x in range(len(data_clean)):
    if data_clean['overall_sentiment'][x]=='very_positive':
        data_clean['overall_sentiment'][x]='positive'
    elif data_clean['overall_sentiment'][x]=='very_negative':
        data_clean['overall_sentiment'][x]='negative'

In [14]:
data_clean['overall_sentiment'].unique()

array(['positive', 'neutral', 'negative'], dtype=object)

In [15]:
order  ={"positive" : 1, "neutral" : 0, "negative" : -1}

In [16]:
data_clean["overall_sentiment"] = data_clean["overall_sentiment"].map(order)

In [17]:
data_clean

,Unnamed: 0,image_name,text_ocr,text_corrected,text_clean,overall_sentiment
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,"[look, friend, lightyear, sohalikut, trend, pl...",1
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,"[best, yearchallenge, completed, le, year, kud...",1
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,"[sam, thorne, follow, follow, saw, everyone, p...",1
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,"[year, challenge, sweet, dee, edition]",1
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,"[year, challenge, filter, hilarious, year, cha...",0
...,...,...,...,...,...,...
6987,6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,Tuesday is Mardi Gras Wednesday is Valentine's...,"[tuesday, mardi, gras, wednesday, valentine, f...",0
6988,6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,"[must, watch, movie, iti, chennai, meme, maana...",0
6989,6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,"[le, talking, planning, soda, junk, food, comp...",1
6990,6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,When I have time is a fantasy. no one has time...,"[time, fantasy, one, time, unless, make, time,...",1


In [18]:
vals = data_clean["overall_sentiment"]

In [19]:
m = len(data_clean['text_clean'])
j = 0
for i in range(0,m):
    data_clean['text_clean'][j] = TreebankWordDetokenizer().detokenize( data_clean['text_clean'].iloc[j])
    j+= 1

In [20]:
data_clean = data_clean.sort_values("image_name", axis = 0, ascending = True)
data_clean = data_clean.reset_index()
data_clean

,index,Unnamed: 0,image_name,text_ocr,text_corrected,text_clean,overall_sentiment
0,0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,look friend lightyear sohalikut trend play yea...,1
1,9,9,image_10.png,FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMP...,FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMP...,facebook year challenge ploy simple meme mo ex...,1
2,99,99,image_100.jpg,Drink water you may not meme generator.com,Drink water you may not meme-generator.com,drink water may memegeneratorcom,0
3,999,999,image_1000.png,RT @BehindScenesPic: Martin Scorsese and Leona...,RT @BehindScenesPic: Martin Scorsese and Leona...,martin scorsese leonardo dicaprio set departed,0
4,1000,1000,image_1001.png,Russian Leonardo DiCaprio omg,Russian Leonardo DiCaprio omg,russian leonardo dicaprio omg,1
...,...,...,...,...,...,...,...
6987,994,994,image_995.jpeg,DON'T LET YOUR DREAMS BE DREAMS! NOTHING IS IM...,DON'T LET YOUR DREAMS BE DREAMS! NOTHING IS IM...,dont let dream dream nothing impossible clp co...,1
6988,995,995,image_996.jpg,A TOAST TO THE MEMORY OF MY WORKOUT... CUZ I'M...,A TOAST TO THE MEMORY OF MY WORKOUT... CUZ I'M...,toast memory workout cuz im murder cffy imgfli...,-1
6989,996,996,image_997.jpg,RARE PICTURE OF LEONARDO DICAPRIO HOLDINGOSCAR...,RARE PICTURE OF LEONARDO DICAPRIO HOLDINGOSCAR...,rare picture leonardo dicaprio holdingoscar st...,1
6990,997,997,image_998.jpg,RIP WORLD'S LONGEST MEME 415 STARECAT.COM,RIP WORLD'S LONGEST MEME STARECAT.COM,rip world longest meme starecatcom,1


In [21]:
tf = TfidfVectorizer(max_df = 0.5)
tfidf = tf.fit_transform(data_clean["text_clean"])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(tfidf ,vals, test_size = 0.3, random_state = 104, shuffle = True)

# Random Forest Classifier


In [25]:
text_classifier = RandomForestClassifier()
text_classifier.fit(X_train,y_train)
Predict = text_classifier.predict(X_test)
pickle.dump(text_classifier, open("rfc.pkl" , "wb"))
print("Confusion matrix", classification_report(y_test,Predict ))
print("Accuracy:", metrics.accuracy_score (y_test, Predict)*100)
print("F1-Score:", metrics.f1_score(y_test, Predict,average=None).mean()*100)
print("F1-Score macro:", metrics.f1_score(y_test, Predict,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

          -1       0.15      0.02      0.03       161
           0       0.27      0.08      0.13       666
           1       0.60      0.88      0.71      1271

    accuracy                           0.56      2098
   macro avg       0.34      0.33      0.29      2098
weighted avg       0.46      0.56      0.48      2098

Accuracy: 56.29170638703527
F1-Score: 29.129994730109583
F1-Score macro: 29.129994730109583


# Decision Tree


In [28]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
pre_y=clf.predict(X_test)
pickle.dump(clf, open("clf.pkl" , "wb"))
print("Confusion matrix", classification_report(y_test,pre_y))
print('Accuracy:', accuracy_score(y_test,pre_y)*100)
print("F1-Score:", metrics.f1_score(y_test, pre_y,average = None).mean()*100) 
print("F1-Score macro:", metrics.f1_score(y_test,pre_y,average = 'macro')*100)

Confusion matrix               precision    recall  f1-score   support

          -1       0.07      0.06      0.06       161
           0       0.33      0.28      0.30       666
           1       0.61      0.67      0.64      1271

    accuracy                           0.50      2098
   macro avg       0.33      0.33      0.33      2098
weighted avg       0.48      0.50      0.49      2098

Accuracy: 49.85700667302193
F1-Score: 33.32389324302662
F1-Score macro: 33.32389324302662


# KNN

In [29]:
KNC = KNeighborsClassifier()
KNC.fit(X_train, y_train)
pr = KNC.predict(X_test)
pickle.dump(KNC, open("knn.pkl" , "wb"))
print("Confusion matrix", classification_report(y_test,pr ))
print("Accuracy:", metrics.accuracy_score (y_test, pr)*100)
print("F1-Score:", metrics.f1_score(y_test, pr,average=None).mean()*100)
print("F1-Score Macro:", metrics.f1_score(y_test, pr,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

          -1       0.04      0.01      0.01       161
           0       0.31      0.29      0.30       666
           1       0.60      0.69      0.64      1271

    accuracy                           0.51      2098
   macro avg       0.32      0.33      0.32      2098
weighted avg       0.47      0.51      0.49      2098

Accuracy: 51.04861773117254
F1-Score: 31.866446931800297
F1-Score Macro: 31.866446931800297


# IMAGES DATA

In [30]:
path = r"C:\Users\S H K\Downloads\project\images\\"
get = os.listdir(path)
imgs=[]
k = 0
for n in get:
    var = skimage.transform.resize(imread(path + n, as_gray = True),(100,100))
    imgs.append(var)
    k += 1

In [31]:
Dataframe = {'Name' : get, 'Images' : imgs}

In [32]:
df1 = pd.DataFrame(Dataframe)
df1

,Name,Images
0,image_1.jpg,"[[0.4128766205400558, 0.41224572579398944, 0.4..."
1,image_10.png,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
2,image_100.jpg,"[[0.116763438293343, 0.11306334238026056, 0.10..."
3,image_1000.png,"[[0.2846914682299253, 0.2868378261296744, 0.28..."
4,image_1001.png,"[[0.9715638862606675, 0.9714675096530226, 0.97..."
...,...,...
6987,image_995.jpeg,"[[0.5911205309299019, 0.5926397697532422, 0.59..."
6988,image_996.jpg,"[[0.3494892657358355, 0.42102657341101696, 0.4..."
6989,image_997.jpg,"[[0.8424193718350534, 0.8754715828880489, 0.64..."
6990,image_998.jpg,"[[0.9999996032786497, 0.9999955080211308, 0.99..."


In [33]:
df1['Flatten']=0
df1['Flatten'][0]=np.array([1,2], dtype=object)

In [34]:
i=0

for i in range(len(df)):
    x = np.array(df1['Images'][i])
    df1['Flatten'][i]=x.flatten()
    i=i+1

In [35]:
df2 = df1.sort_values('Name', ascending = True)

In [36]:
df2['labels'] = data_clean['overall_sentiment']

In [37]:
df2

,Name,Images,Flatten,labels
0,image_1.jpg,"[[0.4128766205400558, 0.41224572579398944, 0.4...","[0.4128766205400558, 0.41224572579398944, 0.41...",1
1,image_10.png,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1
2,image_100.jpg,"[[0.116763438293343, 0.11306334238026056, 0.10...","[0.116763438293343, 0.11306334238026056, 0.108...",0
3,image_1000.png,"[[0.2846914682299253, 0.2868378261296744, 0.28...","[0.2846914682299253, 0.2868378261296744, 0.283...",0
4,image_1001.png,"[[0.9715638862606675, 0.9714675096530226, 0.97...","[0.9715638862606675, 0.9714675096530226, 0.971...",1
...,...,...,...,...
6987,image_995.jpeg,"[[0.5911205309299019, 0.5926397697532422, 0.59...","[0.5911205309299019, 0.5926397697532422, 0.598...",1
6988,image_996.jpg,"[[0.3494892657358355, 0.42102657341101696, 0.4...","[0.3494892657358355, 0.42102657341101696, 0.45...",-1
6989,image_997.jpg,"[[0.8424193718350534, 0.8754715828880489, 0.64...","[0.8424193718350534, 0.8754715828880489, 0.645...",1
6990,image_998.jpg,"[[0.9999996032786497, 0.9999955080211308, 0.99...","[0.9999996032786497, 0.9999955080211308, 0.999...",1


In [38]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [39]:
y_img = le.fit_transform(df2['labels'])

In [40]:
X_img = df2['Flatten']

In [41]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X_img ,y_img, test_size = 0.3, random_state = 104, shuffle = True)

# SVC CLASSIFIER

In [44]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(list(X1_train), list(y1_train))
pred_y = svclassifier.predict(list(X1_test))
pickle.dump(svclassifier, open("svc.pkl" , "wb"))
print("Confusion matrix", classification_report(list(y1_test),pred_y ))
print("Accuracy:", metrics.accuracy_score (list(y1_test), pred_y)*100)
print("F1-Score:", metrics.f1_score(list(y1_test), pred_y,average=None).mean()*100)
print("F1-Score macro:", metrics.f1_score(list(y1_test), pred_y,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

           0       0.10      0.14      0.11       184
           1       0.35      0.33      0.34       698
           2       0.58      0.56      0.57      1216

    accuracy                           0.44      2098
   macro avg       0.34      0.34      0.34      2098
weighted avg       0.46      0.44      0.45      2098

Accuracy: 44.37559580552908
F1-Score: 33.963260465894734
F1-Score macro: 33.963260465894734


# LOGISTIC REGRESSION

In [46]:
lr = LogisticRegression(solver='sag').fit(list(X1_train),list(y1_train))
pred2 = lr.predict(list(X1_test))
pickle.dump(lr, open("lr.pkl" , "wb"))
print("Confusion matrix", classification_report(list(y1_test),list(pred2)))
print('Accuracy:', accuracy_score(list(y1_test),pred2)*100)
print("F1-Score:", metrics.f1_score(list(y1_test), pred2,average=None).mean()*100) 
print("F1-Score macro:", metrics.f1_score(list(y1_test), pred2,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

           0       0.10      0.06      0.08       184
           1       0.34      0.27      0.30       698
           2       0.58      0.68      0.62      1216

    accuracy                           0.49      2098
   macro avg       0.34      0.34      0.33      2098
weighted avg       0.46      0.49      0.47      2098

Accuracy: 48.80838894184938
F1-Score: 33.287198224238765
F1-Score macro: 33.287198224238765


# GAUSSIAN

In [65]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(list(X1_train), list(y1_train))
Pred_e= nb.predict(list(X1_test))
print("Confusion matrix", classification_report(list(y1_test),Pred_e ))
print("Accuracy:", metrics.accuracy_score (list(y1_test), Pred_e)*100)
print("F1-Score:", metrics.f1_score(list(y1_test), Pred_e,average=None).mean()*100)
print("F1-Score macro:", metrics.f1_score(list(y1_test), Pred_e,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

           0       0.09      0.39      0.15       184
           1       0.31      0.21      0.25       698
           2       0.57      0.41      0.48      1216

    accuracy                           0.34      2098
   macro avg       0.33      0.34      0.29      2098
weighted avg       0.44      0.34      0.37      2098

Accuracy: 34.0324118207817
F1-Score: 29.29722071190934
F1-Score macro: 29.29722071190934


In [66]:
diction = {"RFC" : Predict , "DT" : pre_y, "KNN" : pr, "SVC" : pred_y , "LR" : pr , "GAUS" : pred_e}

In [72]:
df4 = pd.DataFrame(diction)
df4.head()

,RFC,DT,KNN,SVC,LR,GAUS
0,1,-1,1,1,2,2
1,1,1,1,1,1,2
2,1,1,1,2,2,2
3,1,0,0,2,2,2
4,1,0,0,2,2,2


In [68]:
new_prediction = df4.mode(axis = 1)

In [69]:
new_prediction = new_prediction[0]

In [70]:
new_prediction

0       1.0
1       1.0
2       1.0
3       2.0
4       2.0
       ... 
2093    1.0
2094    1.0
2095    2.0
2096    0.0
2097    1.0
Name: 0, Length: 2098, dtype: float64

In [71]:
print("Confusion matrix", classification_report(y1_test,new_prediction))
print("Accuracy:", metrics.accuracy_score (y1_test, new_prediction)*100)
print("F1-Score:", metrics.f1_score(y1_test, new_prediction,average=None).mean()*100)
print("F1-Score macro:", metrics.f1_score(y1_test, new_prediction,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         0
         0.0       0.08      0.09      0.08       184
         1.0       0.33      0.63      0.43       698
         2.0       0.58      0.26      0.36      1216

    accuracy                           0.37      2098
   macro avg       0.25      0.24      0.22      2098
weighted avg       0.45      0.37      0.36      2098

Accuracy: 36.98760724499523
F1-Score: 21.956433460958348
F1-Score macro: 21.956433460958348


In [74]:
from sklearn.ensemble import VotingClassifier
vote = VotingClassifier(estimators = [("RFC" , text_classifier) , ("DT" , clf), ("KNN" , KNC), ("SVC" ,svclassifier) , ("LR" , lr) , ("GAUS" , nb)])

In [76]:
vote.fit(list(X1_train), list(y1_train))
pred_v= vote.predict(list(X1_test))
print("Confusion matrix", classification_report(list(y1_test),pred_v ))
print("Accuracy:", metrics.accuracy_score (list(y1_test), pred_v)*100)
print("F1-Score:", metrics.f1_score(list(y1_test), pred_v,average=None).mean()*100)
print("F1-Score macro:", metrics.f1_score(list(y1_test), pred_v,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

           0       0.09      0.07      0.07       184
           1       0.35      0.18      0.23       698
           2       0.58      0.77      0.66      1216

    accuracy                           0.51      2098
   macro avg       0.34      0.34      0.32      2098
weighted avg       0.46      0.51      0.47      2098

Accuracy: 51.00095328884652
F1-Score: 32.3659884666753
F1-Score macro: 32.3659884666753
